In [1]:
import pandas as pd
import duckdb
from pathlib import Path
# --- Importe suas classes de sistema ---
from power.systems import B3EOL, IEEE118EOL, B6L8EOL
import numpy as np

In [3]:
results_path = Path("results_ramping/2025-10-20_02-37-52")

df_angulos = pd.read_parquet(f"{results_path}/angulos.parquet")
df_cargas_individuais = pd.read_parquet(f"{results_path}/cargas_individuais.parquet")
df_corte_carga = pd.read_parquet(f"{results_path}/corte_carga.parquet")
df_corte_carga_detalhado = pd.read_parquet(f"{results_path}/corte_carga_detalhado.parquet")
df_curtailment_detalhado = pd.read_parquet(f"{results_path}/curtailment_detalhado.parquet")
df_fluxo = pd.read_parquet(f"{results_path}/fluxo.parquet")
df_geracao = pd.read_parquet(f"{results_path}/geracao.parquet")
df_limites_angulo = pd.read_parquet(f"{results_path}/limites_angulo.parquet")
df_limites_corte = pd.read_parquet(f"{results_path}/limites_corte.parquet")
df_limites_fluxo = pd.read_parquet(f"{results_path}/limites_fluxo.parquet")
df_limites_geracao = pd.read_parquet(f"{results_path}/limites_geracao.parquet")
df_lmp = pd.read_parquet(f"{results_path}/lmp.parquet")
df_perdas_barra = pd.read_parquet(f"{results_path}/perdas_barra.parquet")
df_perdas_linha = pd.read_parquet(f"{results_path}/perdas_linha.parquet")
df_sumario_geral = pd.read_parquet(f"{results_path}/sumario_geral.parquet")

# --- Primeira Inspeção ---
print("Dados Carregados com Sucesso!")


Dados Carregados com Sucesso!


In [5]:
import pandas as pd
import numpy as np

# Supondo que df_geracao já foi carregado e tem as colunas: 
# ['sistema', 'cenario', 'contingencia', 'gerador_id', 'geracao_mw']

# --- 1. PRÉ-PROCESSAMENTO CRONOLÓGICO (Obrigatório) ---
# A coluna 'contingencia' é apenas o ID do cenário, mas precisamos dela no groupby para isolar a série.
df_geracao['contingencia'] = df_geracao['contingencia'].astype(str)
df_geracao['cenario'] = pd.to_numeric(df_geracao['cenario'])

# CRUCIAL: Ordena o DataFrame. O .shift() vai ocorrer sequencialmente de C0 a C9.
df_geracao.sort_values(
    by=['sistema', 'gerador_id', 'cenario'], # Contingência removida daqui!
    inplace=True
)

# --- 2. CÁLCULO DOS DELTAS E AGRUPAMENTO CONTÍNUO ---

# 2.1. Cria o Delta P (Pi - P_i-1)
# O agrupamento agora ignora a coluna 'contingencia'
df_geracao['geracao_anterior_mw'] = df_geracao.groupby(
    ['sistema', 'gerador_id']
)['geracao_mw'].shift(1)
df_geracao['delta_p_mw'] = df_geracao['geracao_mw'] - df_geracao['geracao_anterior_mw']

# 2.2. Define a direção e cria os grupos de rampa contínua
df_geracao['rampa_direcao'] = np.sign(df_geracao['delta_p_mw'].fillna(0))

# O grupo agora é definido APENAS pela mudança de direção dentro do par (Sistema, Gerador)
df_geracao['rampa_grupo'] = (
    (df_geracao['rampa_direcao'] != df_geracao['rampa_direcao'].shift(1))
).cumsum()

# 2.3. Função para calcular o PULO ACUMULADO (Amplitude: Max P - Min P em cada grupo)
def calculate_rampa_acumulada(group):
    return group['geracao_mw'].max() - group['geracao_mw'].min()

rampa_acumulada_serie = df_geracao.groupby('rampa_grupo').apply(calculate_rampa_acumulada)
df_geracao['pulo_acumulado_total'] = df_geracao['rampa_grupo'].map(rampa_acumulada_serie)

# 2.4. Isola o MVU Acumulado (para subidas) e o MVD Acumulado (para descidas)
df_geracao['mvu_max_acumulado'] = df_geracao.apply(
    lambda row: row['pulo_acumulado_total'] if row['rampa_direcao'] > 0 else 0, axis=1
)
df_geracao['mvd_max_acumulado'] = df_geracao.apply(
    lambda row: row['pulo_acumulado_total'] if row['rampa_direcao'] < 0 else 0, axis=1
)

# =============================================================
# 3. AGREGAÇÃO FINAL (MVu/MVd Máximo por Gerador)
# =============================================================

# Agregação: Armazenamos o VALOR MÁXIMO da soma acumulada que ocorreu para cada gerador.
# Não há mais contingência no agrupamento!
analise_rampa = df_geracao.groupby(['sistema', 'gerador_id']).agg(
    # Armazena a SOMA ACUMULADA MÁXIMA de subida
    mvu_max_acumulado=('mvu_max_acumulado', 'max'),
    # Armazena a SOMA ACUMULADA MÁXIMA de descida
    mvd_max_acumulado=('mvd_max_acumulado', 'max'),
).reset_index()


# Filtra para remover uso insignificante
analise_rampa_final = analise_rampa[
    (analise_rampa['mvu_max_acumulado'] > 1e-6)
].sort_values(by=['sistema', 'mvu_max_acumulado'], ascending=[True, False])


# --- Relatório Final ---
print("\n" + "="*80)
print(" AVALIAÇÃO DO MÁXIMO PULSO DE RAMPA ACUMULADA (MVu e MVd) POR GERADOR ")
print(" (Representa a maior exigência de rampa sustentada na sequência de cenários) ")
print("="*80)

for sistema, df_sistema in analise_rampa_final.groupby('sistema'):
    print(f"\n--- SISTEMA: {sistema} ---")
    
    # Ordena o ranking pela exigência máxima acumulada de subida (MVu Acumulado)
    ranking_sistema = df_sistema.sort_values(by='mvu_max_acumulado', ascending=False)
    
    print("Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):")
    print(ranking_sistema[[
        'gerador_id', 
        'mvu_max_acumulado',
        'mvd_max_acumulado',
    ]].to_string(index=False))


 AVALIAÇÃO DO MÁXIMO PULSO DE RAMPA ACUMULADA (MVu e MVd) POR GERADOR 
 (Representa a maior exigência de rampa sustentada na sequência de cenários) 

--- SISTEMA: B3_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
          2           2.411097            0.06154

--- SISTEMA: B6L8_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
          5          35.739877           0.000000
          2          14.000000          14.000000
          3          12.000000          12.000000
          4          11.085937           0.000000
          6          10.676840           4.376954

--- SISTEMA: IEEE_118_Eolic ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
         20          15.997347           2.690854
         19          13.213431         

C:\Users\gabriel.halfeld\AppData\Local\Temp\ipykernel_9324\1771665692.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rampa_acumulada_serie = df_geracao.groupby('rampa_grupo').apply(calculate_rampa_acumulada)


In [6]:
for sistema, df_sistema in analise_rampa_final.groupby('sistema'):
    print(f"\n--- SISTEMA: {sistema} ---")
    
    # Ordena o ranking pela exigência máxima acumulada de subida (MVu Acumulado)
    ranking_sistema = df_sistema.sort_values(by='mvd_max_acumulado', ascending=False)
    
    print("Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):")
    print(ranking_sistema[[
        'gerador_id', 
        'mvu_max_acumulado',
        'mvd_max_acumulado',
    ]].to_string(index=False))


--- SISTEMA: B3_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
          2           2.411097            0.06154

--- SISTEMA: B6L8_EOLIC ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
          2          14.000000          14.000000
          3          12.000000          12.000000
          6          10.676840           4.376954
          5          35.739877           0.000000
          4          11.085937           0.000000

--- SISTEMA: IEEE_118_Eolic ---
Ranking dos Geradores com MAIOR EXIGÊNCIA MÁXIMA ACUMULADA DE RAMPA (MW):
 gerador_id  mvu_max_acumulado  mvd_max_acumulado
          9          11.134030          17.590250
         10           6.549540          11.576150
         18           1.380178           9.834211
         16           4.271023           8.821323
         17           0.341724         